# Final Project

Big Data Platforms </br>Nic Carlson

In [1]:
# Date notebook was last run
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Mon, 15 March 2021 19:23:18'

## Setup environment

In [2]:
#Ensure we are using the right kernel
print (sc.version)

3.0.1


In [3]:
# Environment
from google.cloud import storage
import io
import os
import subprocess
import shutil
warnings.filterwarnings(action='ignore')

import pyspark.sql.functions as F
import pandas as pd
import re
import json
from itertools import combinations, takewhile
import collections

# Data analysis and visualization
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from itertools import compress 
import seaborn as sns
import matplotlib.pyplot as plt

# Data selection and transformation
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Pulling Tweets

In [4]:
directory = 'gs://msca-bdp-tweets/Tweets/'
file = 'tweets201706221015.json'
# file = '*.json'
path = directory + file

In [5]:
!hadoop fs -du -s -h 'gs://msca-bdp-tweets/Tweets/'

2.2 T  2.2 T  gs://msca-bdp-tweets/Tweets


In [5]:
#SAMPLE
# %time tweets_df = spark.read.json(path)

CPU times: user 3.78 ms, sys: 947 µs, total: 4.72 ms
Wall time: 9.71 s


In [ ]:
#Read ALLLLLLL THE TWEEETS!!!!
# DON"T DO THIS AGAIN UNTIL YOU"RE READY TO FILTER AND SAVE THE DATA. TOOK 26 MINUTES
%time tweets_df = spark.read.json(directory)

CPU times: user 226 ms, sys: 70.5 ms, total: 296 ms
Wall time: 25min


In [ ]:
print("Done!")

Done!


In [ ]:
# %time tweets_df.count()

### Exploring Data

In [26]:
# tweets_df.printSchema()

In [8]:
tweets_df.limit(3)

contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,limit,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
null,null,Thu Jun 22 14:15:...,null,"[[],, [], [[stanf...",null,null,0,false,low,null,877892686513975296,877892686513975296,null,null,null,null,null,false,en,null,null,false,null,null,null,0,false,"[,, Thu Jun 22 14...","<a href=""http://t...",RT @ArkansasBlog:...,1498140901579,false,"[false, Sun Jun 1...",null
null,null,Thu Jun 22 14:15:...,"[0, 140]","[[],, [], [[twitt...",null,"[[0, 129], [[[[11...",0,false,low,null,877892684756566016,877892684756566016,null,null,null,null,null,false,en,null,null,false,null,null,null,0,false,null,"<a href=""https://...",IL Healthcare Tru...,1498140901160,true,"[false, Fri Jun 2...",null
null,null,Thu Jun 22 14:15:...,"[0, 140]","[[],, [], [[twitt...",null,"[[0, 129], [[[[11...",0,false,low,null,877892686744698882,877892686744698882,null,null,null,null,null,false,en,null,null,false,null,null,null,0,false,null,"<a href=""https://...",IL Healthcare Tru...,1498140901634,true,"[false, Thu Feb 0...",null


## Complete thorough EDA to identify which variables you can use to profile the Twitterers

### Features selection process (and post-selection filtering for sparse coluns)

My process went as such:
 
 - I read through Twitter's documentation to get a sense of the Tweet metadata json file
 - Then read through the schema and noted down the features I thought might be interesting/helpful
 - Afterwards, I pulled all of these into a SparkDF and for each column, calculated the percent of cells that had data
 - From there, I selected rows that were at least 80% filled

In [11]:
# mydf = relevantTweets.selectExpr(['contributors as contributors1', 'coordinates.coordinates as tweet_coordinates', 'created_at as tweet_created_at', 'entities.hashtags as tweet_hashtags', 'entities.media.type as media_type', 'entities.media.media_url', 'entities.user_mentions', 'favorite_count as tweet_favorite_count', 'favorited as tweet_favorited',
#                          'geo.coordinates as tweet_geo_coordinates', 'id as tweet_id', 'in_reply_to_screen_name as original_screen_name', 'in_reply_to_status_id as original_tweet_id', 'is_quote_status', 'lang as tweet_lang', 'place.bounding_box.coordinates as reply_coordinates', 
#                          'place.country as tweet_country', 'place.country_code as tweet_country_code', 'place.name as tweet_place_name', 'possibly_sensitive',
#                          'quoted_status.entities.hashtags.text as quoted_hastags', 'retweet_count', 'retweeted', 'retweeted_status', 'retweeted_status.coordinates.coordinates as retweeted_coordinates', 'retweeted_status.entities', 
#                          'retweeted_status.entities.hashtags as retweeted_status_hashtags', 'retweeted_status.contributors as retweeted_status_contributors', 'retweeted_status.coordinates.coordinates as retweet_coordinates', 'retweeted_status.created_at as retweet_created_at', 
#                          'retweeted_status.extended_tweet.full_text', 'retweeted_status.favorite_count as retweet_favorited_count', 'retweeted_status.favorited as retweet_favorited', 
#                          'retweeted_status.id as retweet_id', 'retweeted_status.lang as retweet_lang', 'retweeted_status.place.country as retweet_country', 
#                          'retweeted_status.place.country_code as retweeted_country_code', 'retweeted_status.place.name as retweet_place_name', 'retweeted_status.quoted_status retweet_quoted', 'retweeted_status.retweet_count as retweet_retweeted_count', 'retweeted_status.retweeted as retweet_retweeted', 
#                          'retweeted_status.text as retweet_text', 'retweeted_status.user.id as retweet_user_id', 'retweeted_status.user.followers_count as retweeted_status_followers_count', 'retweeted_status.user.name as retweet_user_name', 'retweeted_status.user.lang as retweet_user_lang', 
#                          'retweeted_status.user.screen_name as retweet_screen_name', 'quoted_status as tweet_quoted', 'quoted_status.contributors as quoted_status_contributors', 'source', 'text', 
#                          'timestamp_ms', 'truncated', 'user.created_at as user_created_at', 'user.description', 'user.favourites_count', 'user.follow_request_sent', 'user.followers_count as user_followers_count', 'user.following', 'user.friends_count', 
#                          'user.geo_enabled as user_geo_enabled', 'user.id as user_id', 'user.is_translator as user_is_translator', 'user.lang as user_lang', 'user.listed_count as user_listed_count', 'user.location as user_location', 'user.name as user_name', 'user.notifications as user_notifications', 'user.screen_name as user_screen_name', 'user.time_zone as user_time_zone', 
#                          'user.url as user_url', 'user.utc_offset as user_utc_offset', 'user.verified as user_verified'])

# mydf.limit(1)

### Using test data, I've removed features where value is null for >95% of the records

In [12]:
# df_agg = mydf.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in mydf.columns]).toPandas().T

In [13]:
# # Print out percentage of columns that are null to help determine which columns to remove. For now, I'll just remove columns that are 80% blank or worse

# pd.set_option('display.max_columns', df_agg.shape[0]+1)

# dfKeep = df_agg / mydf.count()
# dfKeep.columns = ['Percent']
# dfKeep.T

,contributors1,tweet_coordinates,tweet_created_at,tweet_hashtags,media_type,media_url,user_mentions,tweet_favorite_count,tweet_favorited,tweet_geo_coordinates,tweet_id,original_screen_name,original_tweet_id,is_quote_status,tweet_lang,reply_coordinates,tweet_country,tweet_country_code,tweet_place_name,possibly_sensitive,quoted_hastags,retweet_count,retweeted,retweeted_status,retweeted_coordinates,entities,retweeted_status_hashtags,retweeted_status_contributors,retweet_coordinates,retweet_created_at,full_text,retweet_favorited_count,retweet_favorited,retweet_id,retweet_lang,retweet_country,retweeted_country_code,retweet_place_name,retweet_quoted,retweet_retweeted_count,retweet_retweeted,retweet_text,retweet_user_id,retweeted_status_followers_count,retweet_user_name,retweet_user_lang,retweet_screen_name,tweet_quoted,quoted_status_contributors,source,text,timestamp_ms,truncated,user_created_at,description,favourites_count,follow_request_sent,user_followers_count,following,friends_count,user_geo_enabled,user_id,user_is_translator,user_lang,user_listed_count,user_location,user_name,user_notifications,user_screen_name,user_time_zone,user_url,user_utc_offset,user_verified
Percent,1.0,0.979798,0.0,0.0,0.888889,0.888889,0.0,0.0,0.0,0.979798,0.0,0.919192,0.929293,0.0,0.0,0.959596,0.959596,0.959596,0.959596,0.353535,0.949495,0.0,0.0,0.727273,1.0,0.727273,0.727273,1.0,1.0,0.727273,0.89899,0.727273,0.727273,0.727273,0.727273,0.989899,0.989899,0.989899,0.989899,0.727273,0.727273,0.727273,0.727273,0.727273,0.727273,0.727273,0.727273,0.949495,1.0,0.0,0.0,0.0,0.0,0.0,0.191919,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.373737,0.0,1.0,0.0,0.313131,0.484848,0.313131,0.0


In [ ]:
# Selecting only relevant columns that also are at least >80% non-null
mydf = tweets_df.selectExpr(['user.id as user_id', 'user.friends_count', 'user.location as user_location', 'user.name as user_name', 'user.screen_name as user_screen_name',
    'user.created_at as user_created_at', 'user.description', 'user.favourites_count', 'user.followers_count as user_followers_count',
    'favorite_count as tweet_favorite_count', 'id as tweet_id', 'created_at as tweet_created_at','text', 'lang as tweet_lang', 
    'retweeted_status.retweet_count as retweeted_count','retweeted_status.created_at as retweet_created_at', 
    'retweeted_status.user.followers_count as retweeted_status_followers_count', 'retweeted_status.user.name as retweet_user_name', 'retweeted_status.user.location as retweet_user_location'])

# Adding Tweet text length, as this might be an interesting feature later on, and I'd like to do this while in Spark
mydf = mydf.withColumn('length_of_text', F.length('text'))

In [ ]:
print("Done!")

Done!


In [11]:
# mydf.limit(5)

user_id,friends_count,user_location,user_name,user_screen_name,user_created_at,description,favourites_count,user_followers_count,tweet_favorite_count,tweet_id,tweet_created_at,text,tweet_lang,retweeted_count,retweet_created_at,retweeted_status_followers_count,retweet_user_name,retweet_user_location,length_of_text
155078285,2577,"Pine Bluff, Arkansas",Lake Man,andyrosebrook,Sun Jun 13 02:13:...,Secular humanist ...,56574,1554,0,877892686513975296,Thu Jun 22 14:15:...,RT @ArkansasBlog:...,en,1,Thu Jun 22 14:05:...,25073,Arkansas Blog,"Little Rock, AR",129
7159992,87,Chicago,Annie Williams,annoir,Fri Jun 29 20:27:...,Whatever you can ...,167,22,0,877892684756566016,Thu Jun 22 14:15:...,IL Healthcare Tru...,en,null,null,null,null,null,140
827188692766834688,231,null,Indivisible9IL,indivisible9IL,Thu Feb 02 16:15:...,null,111,264,0,877892686744698882,Thu Jun 22 14:15:...,IL Healthcare Tru...,en,null,null,null,null,null,140
831319111414210560,267,"Illinois, USA",Indivisible Illinois,IndivisibleIL,Tue Feb 14 01:48:...,Progressive activ...,1626,1027,0,877892689118715904,Thu Jun 22 14:15:...,IL Healthcare Tru...,en,null,null,null,null,null,140
39984598,960,null,ISMG Network News,ISMG_News,Thu May 14 12:40:...,All news #infosec...,119,2395,0,877892690242936834,Thu Jun 22 14:15:...,Day 1 of Chicago ...,en,null,null,null,null,null,136


## After I've narrowed down only the most important columns in my Spark DataFrame, I'll identify tweets related to UChicago and 3 universities of your choice that have sufficient tweeter activity.

I'm doing it in this order because this filtering actually requires an action against the dataframe, so by selecting only relevant columns first, this will be much more efficient.

I'm pulling tweets for uChicago (inclusive of sister schools), Northwestern, Harvard, and Yale. In this case, I wanted to choose other highly-ranked schools that have both undergrad and graduate programs of similar caliber

In [11]:
# Building out phrase list for uChicago
# uChicagoPresidentDeansDirectorsStaff = ['Robert J. Zimmer', 'Robert Zimmer', 'Zimmer', 'Ka Yee C. Lee', 'Katie Callow-Wright', 'Kenneth S. Polonsky', 'Kenneth Polonsky', 'Lori Berko', 'Derek R. B. Douglas', 'Derek Douglas', 'Sharon Marine', 'James G. Nondorf', 'James Nondorf', 'Juan de Pablo', 'Paul Rand', 'Michele Rasmussen', 'Ivan Samstein', 'Mark A. Schmid', 'Mark Schmid', 'Kim Taylor', 'Katherine Baicker', 'John W. Boyer', 'John Boyer', 'Deborah Gorman-Smith', 'Thomas J. Miles', 'David Nirenberg', 'Angela V. Olinto', 'Angela Olinto', 'Emily Lynn Osborn', 'Emily Osborn', 'Kenneth S. Polonsky', 'Kenneth Polonsky', 'Madhav V. Rajan', 'Madhav Rajan', 'Anne Walters Robertson', 'Anne Robertson', 'Matthew Tirrell', 'Amanda Woodward', 'Paul Kearns', 'Nigel Lockyer', 'Nipam Patel', 'Bridget Allen', 'Alice J. McLean', 'Becky Rhode', 'Camille Morgan', 'Courtney Davis Curtis', 'Dee Steppes', 'Duane Davis', 'Leigh Breslau ', 'Linda McLarnan', 'Maria Romanucci', 'Mark Brokenshire', 'Marlena Harmon', 'Megan Harris', 'Nicholas Moore', 'Ruth OBrien', 'Sidra Malik', 'Susan Huie', 'Susan Popa', 'Toniann Liotta', 'Tracy McCabe','A.B. Yehoshua ', 'A.K. Ramanujan ', 'Aaron Director ', 'Abraham Adrian Albert', 'Adam Zagajewski ', 'Akira Iriye ', 'Albert A. Michelson ', 'Albert J. Libchaber ', 'Albert Rees ', 'Albert Wohlstetter ', 'Alberto Calderón ', 'Alexander A. Beilinson', 'Alexander Razborov ', 'Alfred Radcliffe-Brown ', 'Allan Bloom ', 'Amy Judith Kass ', 'André Weil ', 'Andrei Okounkov ', 'Andrew M. Davis ', 'Anthony Kenny ', 'Antoni Zygmund ', 'Antonin Scalia ', 'Arjun Appadurai ', 'Arnaldo Momigliano ', 'Arnold Davidson ', 'Arthur Byron Coble', 'Arthur Compton ', 'Barack Obama ', 'Bernadotte E. Schmitt ', 'Bernard Roizman ', 'Bertrand Russell ', 'Bruce Cumings ', 'Bruce Lahn ', 'Bruce Lincoln ', 'Burton Dreben ', 'Carl Rogers ', 'Carlos E.M. Wagner ', 'Cass Sunstein ', 'Charles Edward Merriam ', 'Charles Fefferman ', 'Charles Hartshorne ', 'Charles Larmore ', 'Chauncy Harris ', 'Chicago School of literary criticism', 'Chris Blattman ', 'Christian K. Wedemeyer ', 'Chuan He ', 'Clemens C. J. Roothaan ', 'Clifford Ando ', 'Clifford Geertz ', 'Clyde A. Hutchison Jr.', 'Constantin Fasolt ', 'Cornell Fleischer ', 'Dali Yang ', 'Daniel Boorstin ', 'Daniel Friedan ', 'Daniel Quillen ', 'Daniel Sulmasy ', 'Danielle Allen ', 'Daryl Hine ', 'David Bevington ', 'David E. Wellbery ', 'David M. Schneider ', 'David Nirenberg ', 'David Pingree ', 'David Tracy ', 'Diane Wood ', 'Dipesh Chakrabarty ', 'Donald Bogue ', 'Donald Davidson ', 'Douglas Baird ', 'E. H. Moore', 'Edward F. Wente ', 'Edward Sapir ', 'Edward Teller ', 'Edward W. Kolb ', 'Edwin Hubble ', 'Efim Zelmanov ', 'Eleanor Wilner ', 'Elena Kagan ', 'Enrico Fermi ', 'Eric McKitrick ', 'Ernest Lawrence ', 'Eugene Parker ', 'Evelyn M. Kitagawa ', 'Fay-Cooper Cole ', 'Florence B. Seibert ', 'Francesca Rochberg ', 'Frank Easterbrook ', 'Frank Rattray Lillie ', 'Frank Reynolds', 'Frank Wilczek ', 'Fred Donner ', 'Fred M. Donner ', 'Frederic Thrasher ', 'Frederick A. de Armas ', 'Friedrich Hayek ', 'Gary Becker ', 'Geoffrey R. Stone ', 'George Ellery Hale ', 'George Glauberman', 'George Herbert Mead ', 'George Stigler ', 'Gerald Graff ', 'Gerhard Casper ', 'Gerhard Herzberg ', 'Gil Stein ', 'Gilbert Ames Bliss', 'Greg Lawler', 'Gustave E. von Grunebaum ', 'Hannah Arendt ', 'Hans Joas ', 'Hans Küng ', 'Hans Morgenthau ', 'Hans Rothfels ', 'Harold Lasswell ', 'Harold Pollack ', 'Harold Urey ', 'Henry A. Gleason ', 'Henry Paulson ', 'Herrlee G. Creel ', 'Homi K. Bhabha ', 'Howard Taylor Ricketts ', 'Ian Foster', 'Iñigo Manglano-Ovalle ', 'Ioan P. Culianu ', 'Iris Marion Young ', 'Irving Kaplansky', 'Irving Segal', 'Israel Herstein', 'J. A. B. van Buitenen ', 'J. Peter May ', 'Jacob Marschak ', 'Jacqueline Stewart ', 'James A. Robinson ', 'James Boyd White ', 'James Franck ', 'James Hartle ', 'James Hayden Tufts ', 'James Heckman ', 'James Henry Breasted ', 'James M. Redfield ', 'James R. Lawler ', 'James Westfall Thompson ', 'Jan Chiapusso ', 'Jan E. Goldstein ', 'Janet Johnson ', 'Jean-Luc Marion ', 'Jerry Coyne ', 'Jessica Stockholder ', 'Joachim Wach ', 'John A. List ', 'John Carlstrom ', 'John Dewey ', 'John Eaton ', 'John Haugeland ', 'John Hope Franklin ', 'John Hope Franklin ', 'John Keith Moffat ', 'John L. Kelley', 'John Leonard Clive', 'John Maxwell Coetzee ', 'John Mearsheimer ', 'John T. Cacioppo ', 'John W. Boyer ', 'John Woods ', 'Jonathan Lear ', 'Jonathan M. Hall ', 'Joseph Edward Mayer', 'Joseph Kitagawa ', 'Joseph Lykken ', 'K. Aslihan Yener ', 'Karen Uhlenbeck ', 'Karin Knorr-Cetina ', 'Karl Freed ', 'Karl Lashley ', 'Karl Llewellyn ', 'Karl Weintraub ', 'Katherine Baicker ', 'Kenneth Burke ', 'Kenneth Prewitt ', 'Lanny D. Schmidt', 'Lars Hörmander ', 'László Babai ', 'Lauren Berlant ', 'Lawrence Kohlberg ', 'Leela Gandhi ', 'Leo Strauss ', 'Leonard Bloomfield ', 'Leonard Eugene Dickson ', 'Leszek Kołakowski ', 'Lorraine Daston ', 'Lucy Graves Taliaferro', 'Luis Caffarelli ', 'Marcela Carena ', 'Maria Goeppert-Mayer ', 'Maria Spiropulu ', 'Mark Lilla ', 'Mark Siegler ', 'Mark Strand ', 'Marshall G. S. Hodgson ', 'Marshall Sahlins ', 'Marshall Stone', 'Martha McClintock ', 'Martha Nussbaum ', 'Martha Nussbaum ', 'Martha Roth ', 'Martin Kreitman ', 'Martin Marty', 'Matthew Gentzkow ', 'Matthew Stolper ', 'Maynard C. Krueger ', 'Merton H. Miller ', 'Michael Dickinson ', 'Michael H. Schill ', 'Michael S. Turner ', 'Michael Schudson ', 'Michael Silverstein ', 'Michael W. McConnell ', 'Miguel Civil ', 'Milton Friedman ', 'Mircea Eliade ', 'Morton A. Kaplan ', 'Murray Gell-Mann ', 'Muzaffar Alam ', 'Myrtle Bachelder ', 'Naomi Weisstein ', 'Nathaniel Kleitman ', 'Ngô Bảo Châu ', 'Noel Swerdlow ', 'Norman Golb ', 'Norman Maclean ', 'Norman Steenrod ', 'Ole J. Kleppa ', 'Olufunmilayo Olopade ', 'Oskar Bolza', 'Otto Struve ', 'Paul Halmos ', 'Paul Johannes Tillich ', 'Paul Meier ', 'Paul Ricoeur ', 'Paul Sally ', 'Paul Sigler ', 'Peter Dorman ', 'Peter White ', 'Philip Gossett ', 'R. Stephen Berry ', 'Raghuram Rajan', 'Ralph Buchsbaum ', 'Ralph Ellison ', 'Ralph Shapey ', 'Ramón A. Gutiérrez ', 'Raphael Carl Lee ', 'Raven I. McDavid Jr.', 'Richard Epstein ', 'Richard H. Helmholz ', 'Richard Lewontin ', 'Richard Posner ', 'Richard Shweder ', 'Richard T. Antoun ', 'Robert B. Pippin ', 'Robert Bartlett ', 'Robert E. Park ', 'Robert Fogel ', 'Robert K. Ritner ', 'Robert Lee Moore', 'Robert Lucas Jr. ', 'Robert M. Wald ', 'Robert Millikan ', 'Robert Pape ', 'Robert Pinsky ', 'Robert Redfield ', 'Robert S. Mulliken ', 'Robert Soare ', 'Roberta Cooper Ramo ', 'Roger Ebert ', 'Ronald Coase ', 'Ronald Coase ', 'Ronald Wilbert Harris ', 'Rudolf Carnap ', 'Russell Tuttle ', 'Saskia Sassen ', 'Saul Bellow ', 'Saunders Mac Lane ', 'Savas Dimopoulos ', 'Sean M. Carroll ', 'Serge Lang', 'Sewall Wright ', 'Shadi Bartsch ', 'Shannon Lee Dawdy ', 'Sheila Fitzpatrick ', 'Shiing-shen Chern ', 'Shulamit Ran ', 'Stanley Cavell ', 'Stanley Elkins ', 'Stanley Jeyaraja Tambiah ', 'Stephen Shenker ', 'Stephen Smale ', 'Stephen Walt ', 'Steven Levitt ', 'Stuart Rice ', 'Subrahmanyan Chandrasekhar ', 'Susan Gal ', 'Susan Goldin-Meadow ', 'Susan L. Cohn ', 'T. Theodore Fujita ', 'T.S. Eliot ', 'Theda Skocpol ', 'Theodore O. Yntema ', 'Thomas C. Holt ', 'Thomas Chrowder Chamberlin ', 'Thomas Pavel ', 'Thornton Wilder ', 'Thorstein Veblen ', 'Victor Ginzburg ', 'Victor Turner ', 'Vivian Paley ', 'Vladimir Drinfeld ', 'Walter Blair ', 'Walter Kaegi ', 'Walter Kaegi ', 'Walter Lewis Baily Jr.', 'Warder Clyde Allee ', 'Wayne C. Booth ', 'Wendy Doniger ', 'William Hardy McNeill ', 'William I. Thomas ', 'William Julius Wilson ', 'William Lawvere ', 'William R. Polk ', 'Yoichiro Nambu ', 'Zonia Baber ']

# Pulling these separately for sister schools 1) because the number of rows were too large on the full dataset, and 2) to keep these separate in case I just want to keep the focus generally on tweets about the universities instead of specific sub-schools
uChicagoSchools = ['booth', 'Chicago School of literary criticism', 'Argonne National Laboratory', 'Argonne', 'Biological Sciences Division', 'BSD', 'The University of Chicago Booth School of Business', 'The College Crown Family School of Social Work, Policy, and Practice', 'Divinity School', 'Graham School of Continuing Liberal and Professional Studies', 'Graham school', 'The University of Chicago Harris School of Public Policy Studies', 'harris school', 'chicago harris', 'The University of Chicago Law School', 'University of Chicago Law', 'Oriental Institute', 'Physical Sciences Division', 'PSD', 'Pritzker School of Medicine', 'PSM', 'Pritzker School of Molecular Engineering', 'PSME', 'Social Sciences Division', 'PSD', 'AWH', 'Vogel Lab', 'Vogel']

# Building out phrase list for uChicago
uChicago = ['uchicago.edu','uc','uofc','graham','uchicago', 'university of chicago', 'uchicagolaw', 'Phil the Phoenix', 'maroons', 'chicago maroons']
NW = ['Northwestern', 'Northwestern University']
Harvard = ['Harvard', 'Harvard University']
Yale = ['Yale', 'Yale University']

phrases = uChicago
phrasesBroad = uChicagoSchools

phrases = list(map(lambda x:x.lower(),phrases))
phrasesNW = list(map(lambda x:x.lower(),NW))
phrasesHarvard = list(map(lambda x:x.lower(),Harvard))
phrasesYale = list(map(lambda x:x.lower(),Yale))
phrasesBroad = list(map(lambda x:x.lower(),phrasesBroad))

In [12]:
pd.set_option('display.max_colwidth', None)

relevantTweets = mydf
relevantTweets = relevantTweets.withColumn('text', lower(relevantTweets['text']))
relevantTweets = relevantTweets.filter(col('text').rlike('(^|\s)(' + '|'.join(phrases) + ')(\s|$)'))

# NW
relevantTweetsNW = mydf
relevantTweetsNW = relevantTweetsNW.withColumn('text', lower(relevantTweetsNW['text']))
relevantTweetsNW = relevantTweetsNW.filter(col('text').rlike('(^|\s)(' + '|'.join(phrasesNW) + ')(\s|$)'))

# Harvard
relevantTweetsHarvard = mydf
relevantTweetsHarvard = relevantTweetsHarvard.withColumn('text', lower(relevantTweetsHarvard['text']))
relevantTweetsHarvard = relevantTweetsHarvard.filter(col('text').rlike('(^|\s)(' + '|'.join(phrasesHarvard) + ')(\s|$)'))

# Yale
relevantTweetsYale = mydf
relevantTweetsYale = relevantTweetsYale.withColumn('text', lower(relevantTweetsYale['text']))
relevantTweetsYale = relevantTweetsYale.filter(col('text').rlike('(^|\s)(' + '|'.join(phrasesYale) + ')(\s|$)'))

# Broad
relevantTweetsBroad = mydf
relevantTweetsBroad = relevantTweetsBroad.withColumn('text', lower(relevantTweetsBroad['text']))
relevantTweetsBroad = relevantTweetsBroad.filter(col('text').rlike('(^|\s)(' + '|'.join(phrasesBroad) + ')(\s|$)'))

In [ ]:
relevantTweets.count()

534295

In [ ]:
relevantTweetsNW.count()

1975721

In [ ]:
relevantTweetsHarvard.count()

In [ ]:
relevantTweetsYale.count()

In [ ]:
relevantTweetsBroad.count()

In [40]:
# relevantTweets.limit(1)

tweet_created_at,user_mentions,tweet_favorite_count,tweet_favorited,tweet_id,is_quote_status,tweet_lang,retweet_count,retweeted,retweeted_status_contributors,retweet_created_at,retweet_id,retweet_lang,retweet_quoted,retweet_retweeted_count,retweet_retweeted,retweet_text,retweet_user_id,retweeted_status_followers_count,retweet_user_name,retweet_screen_name,source,text,timestamp_ms,truncated,user_created_at,description,favourites_count,user_followers_count,friends_count,user_id,user_is_translator,user_listed_count,user_location,user_name,user_screen_name,user_url,user_verified,length_of_text
Thu Jun 22 14:16:...,"[[16664681, 16664...",0,false,877892987727912960,false,en,0,false,null,null,null,null,null,null,null,null,null,null,null,null,"<a href=""http://t...",@latimes @latimes...,1498140973394,true,Fri Mar 07 05:59:...,Conservatarian. B...,1847,71,250,2376648480,false,3,null,Elephant in room,dzmanliberty,null,false,139


### Export to CSVs for further analysis in Midway

In [ ]:
# #Test writing did not work, so I had to convert to pandas and then export as CSV
bucket_write = 'msca-bdp-students-bucket/shared_data/carlson1858'

mydf.write.format("com.databricks.spark.csv").\
option("header", "true").\
mode('overwrite').\
save('gs://' + bucket_write + '/RelevantTweets.csv')

In [ ]:
# uChicago
%time relevantTweets.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/RelevantTweetsFinalvFINAL.csv')

CPU times: user 17 s, sys: 800 ms, total: 17.8 s
Wall time: 24min 47s


In [ ]:
# uChicago broad sweep
%time relevantTweetsBroad.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/RelevantTweetsBroaderFinal.csv')

CPU times: user 4.07 s, sys: 187 ms, total: 4.26 s
Wall time: 24min 55s


In [ ]:
# Northwestern
%time relevantTweetsNW.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/RelevantTweetsNWFinal.csv')

CPU times: user 1min 2s, sys: 3.13 s, total: 1min 5s
Wall time: 25min 13s


In [ ]:
# Harvard
%time relevantTweetsHarvard.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/RelevantTweetsHarvardFinal.csv')

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


CPU times: user 1min 19s, sys: 3.88 s, total: 1min 22s
Wall time: 26min 24s


In [ ]:
# Yale
%time relevantTweetsYale.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/RelevantTweetsYaleFinal.csv')

CPU times: user 18.6 s, sys: 854 ms, total: 19.5 s
Wall time: 23min 52s


In [ ]:
print("Done")

## Doing the same thing but now with top Twitterers, after running analysis

In [ ]:
print("done")

done


In [ ]:
# Building out phrase list for uChicago
# uChicagoPresidentDeansDirectorsStaff = ['Robert J. Zimmer', 'Robert Zimmer', 'Zimmer', 'Ka Yee C. Lee', 'Katie Callow-Wright', 'Kenneth S. Polonsky', 'Kenneth Polonsky', 'Lori Berko', 'Derek R. B. Douglas', 'Derek Douglas', 'Sharon Marine', 'James G. Nondorf', 'James Nondorf', 'Juan de Pablo', 'Paul Rand', 'Michele Rasmussen', 'Ivan Samstein', 'Mark A. Schmid', 'Mark Schmid', 'Kim Taylor', 'Katherine Baicker', 'John W. Boyer', 'John Boyer', 'Deborah Gorman-Smith', 'Thomas J. Miles', 'David Nirenberg', 'Angela V. Olinto', 'Angela Olinto', 'Emily Lynn Osborn', 'Emily Osborn', 'Kenneth S. Polonsky', 'Kenneth Polonsky', 'Madhav V. Rajan', 'Madhav Rajan', 'Anne Walters Robertson', 'Anne Robertson', 'Matthew Tirrell', 'Amanda Woodward', 'Paul Kearns', 'Nigel Lockyer', 'Nipam Patel', 'Bridget Allen', 'Alice J. McLean', 'Becky Rhode', 'Camille Morgan', 'Courtney Davis Curtis', 'Dee Steppes', 'Duane Davis', 'Leigh Breslau ', 'Linda McLarnan', 'Maria Romanucci', 'Mark Brokenshire', 'Marlena Harmon', 'Megan Harris', 'Nicholas Moore', 'Ruth OBrien', 'Sidra Malik', 'Susan Huie', 'Susan Popa', 'Toniann Liotta', 'Tracy McCabe','A.B. Yehoshua ', 'A.K. Ramanujan ', 'Aaron Director ', 'Abraham Adrian Albert', 'Adam Zagajewski ', 'Akira Iriye ', 'Albert A. Michelson ', 'Albert J. Libchaber ', 'Albert Rees ', 'Albert Wohlstetter ', 'Alberto Calderón ', 'Alexander A. Beilinson', 'Alexander Razborov ', 'Alfred Radcliffe-Brown ', 'Allan Bloom ', 'Amy Judith Kass ', 'André Weil ', 'Andrei Okounkov ', 'Andrew M. Davis ', 'Anthony Kenny ', 'Antoni Zygmund ', 'Antonin Scalia ', 'Arjun Appadurai ', 'Arnaldo Momigliano ', 'Arnold Davidson ', 'Arthur Byron Coble', 'Arthur Compton ', 'Barack Obama ', 'Bernadotte E. Schmitt ', 'Bernard Roizman ', 'Bertrand Russell ', 'Bruce Cumings ', 'Bruce Lahn ', 'Bruce Lincoln ', 'Burton Dreben ', 'Carl Rogers ', 'Carlos E.M. Wagner ', 'Cass Sunstein ', 'Charles Edward Merriam ', 'Charles Fefferman ', 'Charles Hartshorne ', 'Charles Larmore ', 'Chauncy Harris ', 'Chicago School of literary criticism', 'Chris Blattman ', 'Christian K. Wedemeyer ', 'Chuan He ', 'Clemens C. J. Roothaan ', 'Clifford Ando ', 'Clifford Geertz ', 'Clyde A. Hutchison Jr.', 'Constantin Fasolt ', 'Cornell Fleischer ', 'Dali Yang ', 'Daniel Boorstin ', 'Daniel Friedan ', 'Daniel Quillen ', 'Daniel Sulmasy ', 'Danielle Allen ', 'Daryl Hine ', 'David Bevington ', 'David E. Wellbery ', 'David M. Schneider ', 'David Nirenberg ', 'David Pingree ', 'David Tracy ', 'Diane Wood ', 'Dipesh Chakrabarty ', 'Donald Bogue ', 'Donald Davidson ', 'Douglas Baird ', 'E. H. Moore', 'Edward F. Wente ', 'Edward Sapir ', 'Edward Teller ', 'Edward W. Kolb ', 'Edwin Hubble ', 'Efim Zelmanov ', 'Eleanor Wilner ', 'Elena Kagan ', 'Enrico Fermi ', 'Eric McKitrick ', 'Ernest Lawrence ', 'Eugene Parker ', 'Evelyn M. Kitagawa ', 'Fay-Cooper Cole ', 'Florence B. Seibert ', 'Francesca Rochberg ', 'Frank Easterbrook ', 'Frank Rattray Lillie ', 'Frank Reynolds', 'Frank Wilczek ', 'Fred Donner ', 'Fred M. Donner ', 'Frederic Thrasher ', 'Frederick A. de Armas ', 'Friedrich Hayek ', 'Gary Becker ', 'Geoffrey R. Stone ', 'George Ellery Hale ', 'George Glauberman', 'George Herbert Mead ', 'George Stigler ', 'Gerald Graff ', 'Gerhard Casper ', 'Gerhard Herzberg ', 'Gil Stein ', 'Gilbert Ames Bliss', 'Greg Lawler', 'Gustave E. von Grunebaum ', 'Hannah Arendt ', 'Hans Joas ', 'Hans Küng ', 'Hans Morgenthau ', 'Hans Rothfels ', 'Harold Lasswell ', 'Harold Pollack ', 'Harold Urey ', 'Henry A. Gleason ', 'Henry Paulson ', 'Herrlee G. Creel ', 'Homi K. Bhabha ', 'Howard Taylor Ricketts ', 'Ian Foster', 'Iñigo Manglano-Ovalle ', 'Ioan P. Culianu ', 'Iris Marion Young ', 'Irving Kaplansky', 'Irving Segal', 'Israel Herstein', 'J. A. B. van Buitenen ', 'J. Peter May ', 'Jacob Marschak ', 'Jacqueline Stewart ', 'James A. Robinson ', 'James Boyd White ', 'James Franck ', 'James Hartle ', 'James Hayden Tufts ', 'James Heckman ', 'James Henry Breasted ', 'James M. Redfield ', 'James R. Lawler ', 'James Westfall Thompson ', 'Jan Chiapusso ', 'Jan E. Goldstein ', 'Janet Johnson ', 'Jean-Luc Marion ', 'Jerry Coyne ', 'Jessica Stockholder ', 'Joachim Wach ', 'John A. List ', 'John Carlstrom ', 'John Dewey ', 'John Eaton ', 'John Haugeland ', 'John Hope Franklin ', 'John Hope Franklin ', 'John Keith Moffat ', 'John L. Kelley', 'John Leonard Clive', 'John Maxwell Coetzee ', 'John Mearsheimer ', 'John T. Cacioppo ', 'John W. Boyer ', 'John Woods ', 'Jonathan Lear ', 'Jonathan M. Hall ', 'Joseph Edward Mayer', 'Joseph Kitagawa ', 'Joseph Lykken ', 'K. Aslihan Yener ', 'Karen Uhlenbeck ', 'Karin Knorr-Cetina ', 'Karl Freed ', 'Karl Lashley ', 'Karl Llewellyn ', 'Karl Weintraub ', 'Katherine Baicker ', 'Kenneth Burke ', 'Kenneth Prewitt ', 'Lanny D. Schmidt', 'Lars Hörmander ', 'László Babai ', 'Lauren Berlant ', 'Lawrence Kohlberg ', 'Leela Gandhi ', 'Leo Strauss ', 'Leonard Bloomfield ', 'Leonard Eugene Dickson ', 'Leszek Kołakowski ', 'Lorraine Daston ', 'Lucy Graves Taliaferro', 'Luis Caffarelli ', 'Marcela Carena ', 'Maria Goeppert-Mayer ', 'Maria Spiropulu ', 'Mark Lilla ', 'Mark Siegler ', 'Mark Strand ', 'Marshall G. S. Hodgson ', 'Marshall Sahlins ', 'Marshall Stone', 'Martha McClintock ', 'Martha Nussbaum ', 'Martha Nussbaum ', 'Martha Roth ', 'Martin Kreitman ', 'Martin Marty', 'Matthew Gentzkow ', 'Matthew Stolper ', 'Maynard C. Krueger ', 'Merton H. Miller ', 'Michael Dickinson ', 'Michael H. Schill ', 'Michael S. Turner ', 'Michael Schudson ', 'Michael Silverstein ', 'Michael W. McConnell ', 'Miguel Civil ', 'Milton Friedman ', 'Mircea Eliade ', 'Morton A. Kaplan ', 'Murray Gell-Mann ', 'Muzaffar Alam ', 'Myrtle Bachelder ', 'Naomi Weisstein ', 'Nathaniel Kleitman ', 'Ngô Bảo Châu ', 'Noel Swerdlow ', 'Norman Golb ', 'Norman Maclean ', 'Norman Steenrod ', 'Ole J. Kleppa ', 'Olufunmilayo Olopade ', 'Oskar Bolza', 'Otto Struve ', 'Paul Halmos ', 'Paul Johannes Tillich ', 'Paul Meier ', 'Paul Ricoeur ', 'Paul Sally ', 'Paul Sigler ', 'Peter Dorman ', 'Peter White ', 'Philip Gossett ', 'R. Stephen Berry ', 'Raghuram Rajan', 'Ralph Buchsbaum ', 'Ralph Ellison ', 'Ralph Shapey ', 'Ramón A. Gutiérrez ', 'Raphael Carl Lee ', 'Raven I. McDavid Jr.', 'Richard Epstein ', 'Richard H. Helmholz ', 'Richard Lewontin ', 'Richard Posner ', 'Richard Shweder ', 'Richard T. Antoun ', 'Robert B. Pippin ', 'Robert Bartlett ', 'Robert E. Park ', 'Robert Fogel ', 'Robert K. Ritner ', 'Robert Lee Moore', 'Robert Lucas Jr. ', 'Robert M. Wald ', 'Robert Millikan ', 'Robert Pape ', 'Robert Pinsky ', 'Robert Redfield ', 'Robert S. Mulliken ', 'Robert Soare ', 'Roberta Cooper Ramo ', 'Roger Ebert ', 'Ronald Coase ', 'Ronald Coase ', 'Ronald Wilbert Harris ', 'Rudolf Carnap ', 'Russell Tuttle ', 'Saskia Sassen ', 'Saul Bellow ', 'Saunders Mac Lane ', 'Savas Dimopoulos ', 'Sean M. Carroll ', 'Serge Lang', 'Sewall Wright ', 'Shadi Bartsch ', 'Shannon Lee Dawdy ', 'Sheila Fitzpatrick ', 'Shiing-shen Chern ', 'Shulamit Ran ', 'Stanley Cavell ', 'Stanley Elkins ', 'Stanley Jeyaraja Tambiah ', 'Stephen Shenker ', 'Stephen Smale ', 'Stephen Walt ', 'Steven Levitt ', 'Stuart Rice ', 'Subrahmanyan Chandrasekhar ', 'Susan Gal ', 'Susan Goldin-Meadow ', 'Susan L. Cohn ', 'T. Theodore Fujita ', 'T.S. Eliot ', 'Theda Skocpol ', 'Theodore O. Yntema ', 'Thomas C. Holt ', 'Thomas Chrowder Chamberlin ', 'Thomas Pavel ', 'Thornton Wilder ', 'Thorstein Veblen ', 'Victor Ginzburg ', 'Victor Turner ', 'Vivian Paley ', 'Vladimir Drinfeld ', 'Walter Blair ', 'Walter Kaegi ', 'Walter Kaegi ', 'Walter Lewis Baily Jr.', 'Warder Clyde Allee ', 'Wayne C. Booth ', 'Wendy Doniger ', 'William Hardy McNeill ', 'William I. Thomas ', 'William Julius Wilson ', 'William Lawvere ', 'William R. Polk ', 'Yoichiro Nambu ', 'Zonia Baber ']

# Pulling these separately for sister schools 1) because the number of rows were too large on the full dataset, and 2) to keep these separate in case I just want to keep the focus generally on tweets about the universities instead of specific sub-schools
uChicagoInfluencers = ['CyanFour', 'yiizo', 'the_gwella', 'Katja_Thieme', 'YamraaMcNeil']
NWInfluencers = ['idekchia', '_nnajib', 'Christian_B56', 'olanaaaaa','Rosenchild']
HarvardInfluencers = ['wmzafravelasco', 'raybae689', 'jared_secret', 'JacobG88', 'bz80qrlXkWpoBcT']
YaleInfluencers = ['meghanaaa', 'raybae689', 'iambrittania', 'EXFURIBISTA', 'kaylaiko_']

iphrasesuChicago = list(map(lambda x:x.lower(),uChicagoInfluencers))
iphrasesNW = list(map(lambda x:x.lower(),NWInfluencers))
iphrasesHarvard = list(map(lambda x:x.lower(),HarvardInfluencers))
iphrasesYale = list(map(lambda x:x.lower(),YaleInfluencers))

In [ ]:
pd.set_option('display.max_colwidth', None)

uChicagoInfluencers = mydf
uChicagoInfluencers = uChicagoInfluencers.withColumn('user_screen_name', lower(uChicagoInfluencers['user_screen_name']))
uChicagoInfluencers = uChicagoInfluencers.filter(col('user_screen_name').rlike('(^|\s)(' + '|'.join(iphrasesuChicago) + ')(\s|$)'))

# NW
NWInfluencers = mydf
NWInfluencers = NWInfluencers.withColumn('user_screen_name', lower(NWInfluencers['user_screen_name']))
NWInfluencers = NWInfluencers.filter(col('user_screen_name').rlike('(^|\s)(' + '|'.join(iphrasesNW) + ')(\s|$)'))

# Harvard
HarvardInfluencers = mydf
HarvardInfluencers = HarvardInfluencers.withColumn('user_screen_name', lower(HarvardInfluencers['user_screen_name']))
HarvardInfluencers = HarvardInfluencers.filter(col('user_screen_name').rlike('(^|\s)(' + '|'.join(iphrasesHarvard) + ')(\s|$)'))

# Yale
YaleInfluencers = mydf
YaleInfluencers = YaleInfluencers.withColumn('user_screen_name', lower(YaleInfluencers['user_screen_name']))
YaleInfluencers = YaleInfluencers.filter(col('user_screen_name').rlike('(^|\s)(' + '|'.join(iphrasesYale) + ')(\s|$)'))

In [ ]:
# uChicago
%time uChicagoInfluencers.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/InfluencersuChicago.csv')

CPU times: user 530 ms, sys: 73.5 ms, total: 604 ms
Wall time: 21min 53s


In [ ]:
# Northwestern
%time NWInfluencers.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/InfluencersNW.csv')

CPU times: user 555 ms, sys: 94.3 ms, total: 649 ms
Wall time: 21min 43s


In [ ]:
# Harvard
%time HarvardInfluencers.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/InfluencersHarvard.csv')

CPU times: user 650 ms, sys: 83.1 ms, total: 733 ms
Wall time: 21min 34s


In [ ]:
# Harvard
%time YaleInfluencers.toPandas().to_csv('gs://msca-bdp-students-bucket/shared_data/carlson1858/InfluencersYale.csv')

CPU times: user 378 ms, sys: 67.5 ms, total: 446 ms
Wall time: 21min 28s


In [ ]:
print(uChicagoInfluencers.count())
print(HarvardInfluencers.count())
print(NWInfluencers.count())
print(YaleInfluencers.count())

2479
13536
11475
6325
